In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\liuch001\Downloads\sms_spam.csv',sep=',',header=0, names=['label', 'message'])

In [3]:
print(df.shape)
print(df.label.value_counts())

(5559, 2)
ham     4812
spam     747
Name: label, dtype: int64


In [4]:
sum(df.label == 'spam')/len(df.label)

0.13437668645439826

This dataset contains one set of SMS messages in English of 5,559 messages, tagged according being ham (legitimate) or spam. The distribution is a total of 4,812 SMS legitimate messages (86.6%) and a total of 747 (13.4%) spam messages.

In [5]:
sum((df.label == 'spam') & (df.message.str.contains('sex')))/sum(df.message.str.contains('sex'))

0.6666666666666666

In this dataset, for messsages that contains 'sex', 67% of them are spams.

In [6]:
#convert the labels from strings to binary values for our classifier
df['label'] = df.label.map({'ham': 0, 'spam': 1}) 
#convert all characters in the message to lower case
df['message'] = df.message.map(lambda x: x.lower())
#remove any punctuation
df['message'] = df.message.str.replace('[^\w\s]', '')

In [13]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [29]:
import nltk


In [18]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\liuch001\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [19]:
#tokenize the messages into into single words
df['message'] = df['message'].apply(nltk.word_tokenize)

In [20]:
#perform some word stemming. 
# The idea of stemming is to normalize our text for all variations of words carry the same meaning, regardless of the tense
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
df['message'] = df['message'].apply(lambda x: [stemmer.stem(y) for y in x])

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

# This converts the list of words into space-separated strings
df['message'] = df['message'].apply(lambda x: ' '.join(x))

count_vect = CountVectorizer()
counts = count_vect.fit_transform(df['message'])

In [22]:
counts

<5559x8119 sparse matrix of type '<class 'numpy.int64'>'
	with 71911 stored elements in Compressed Sparse Row format>

In [23]:
#transform the data into occurrences, which will be the features that we will feed into our model

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer().fit(counts)
counts = transformer.transform(counts)

In [24]:
counts

<5559x8119 sparse matrix of type '<class 'numpy.float64'>'
	with 71911 stored elements in Compressed Sparse Row format>

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(counts, df['label'], test_size=0.1, random_state=12)

In [26]:
# Multinomial Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(X_train, y_train)

In [27]:
import numpy as np
predicted = model.predict(X_test)
print(np.mean(predicted == y_test))

0.960431654676259


In [28]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predicted))

[[489   1]
 [ 21  45]]
